In [1]:
!python3 -m venv venv
!source venv/bin/activate 
!python3 -m pip install numpy

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 18.2 MB 8.0 MB/s eta 0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [3]:
!python --version

Python 3.9.5


In [69]:
from pyspark import SparkContext, SparkConf 
conf = SparkConf().setAppName("laba 1 KR").setMaster('yarn')

In [81]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType

In [ ]:
sc = SparkContext.getOrCreate(conf = conf)

In [41]:
tripData = sc.textFile("trips.csv")

tripsHeader = tripData.first()
trips = tripData.filter(lambda row: row != tripsHeader).map(lambda row: row.split(",", -1))

Найти велосипед с максимальным временем пробега.

In [43]:
trips.count()

669959

In [44]:
trips.take(2)

[['4576',
  '63',
  '',
  'South Van Ness at Market',
  '66',
  '8/29/2013 14:14',
  'South Van Ness at Market',
  '66',
  '520',
  'Subscriber',
  '94127'],
 ['4607',
  '',
  '8/29/2013 14:42',
  'San Jose City Hall',
  '10',
  '8/29/2013 14:43',
  'San Jose City Hall',
  '10',
  '661',
  'Subscriber',
  '95138']]

In [45]:
from typing import NamedTuple
from datetime import datetime

def initTrip(trips):
    class Trip(NamedTuple):
        trip_id: int
        duration: int
        start_date: datetime
        start_station_name: str
        start_station_id: int
        end_date: datetime
        end_station_name: str
        end_station_id: int
        bike_id: int
        subscription_type: str
        zip_code: str
        
    for trip in trips:
        try:
            yield Trip(                             
             trip_id = int(trip[0]),
             duration = int(trip[1]),
             start_date = datetime.strptime(trip[2], '%m/%d/%Y %H:%M'),
             start_station_name = trip[3],
             start_station_id = int(trip[4]),
             end_date = datetime.strptime(trip[5], '%m/%d/%Y %H:%M'),
             end_station_name = trip[6],
             end_station_id = trip[7],
             bike_id = int(trip[8]),
             subscription_type = trip[9],
             zip_code = trip[10]
            ) 
        except:
            pass

In [77]:
tripsInternal = trips.mapPartitions(initTrip)

In [72]:
trips_by_bike_id = tripsInternal.keyBy(lambda trip: trip.bike_id)
trips_by_bike_id.take(1)

[(48,
  Trip(trip_id=4130, duration=71, start_date=datetime.datetime(2013, 8, 29, 10, 16), start_station_name='Mountain View City Hall', start_station_id=27, end_date=datetime.datetime(2013, 8, 29, 10, 17), end_station_name='Mountain View City Hall', end_station_id='27', bike_id=48, subscription_type='Subscriber', zip_code='97214'))]

In [48]:
trips_by_duration = trips_by_bike_id.mapValues(lambda trip: trip.duration)

In [49]:
trips_by_duration.groupByKey().mapValues(sum).top(1,  key=lambda x: x[1])

[(535, 18611693)]

In [50]:
t = trips_by_duration.groupByKey().mapValues(sum)

In [51]:
%%time
m = t.map(lambda x: x).top(1, key=lambda x: x[1])

CPU times: user 15.2 ms, sys: 5.41 ms, total: 20.6 ms
Wall time: 17.7 s


In [52]:
print(f"Id: {m[0][0]}, max of duration: {int(m[0][1])/60} min")

Id: 535, max of duration: 310194.88333333336 min


Найти наибольшее геодезическое расстояние между станциями.

In [53]:
def initStation(stations):
    class Station(NamedTuple):
        station_id: int
        name: str
        lat: float
        long: float
        dockcount: int
        landmark: str
        installation: str
    
    for station in stations:
        yield Station(
            station_id = int(station[0]),
            name = station[1],
            lat = float(station[2]),
            long = float(station[3]),
            dockcount = int(station[4]),
            landmark = station[5],
            installation = datetime.strptime(station[6], '%m/%d/%Y')
        )

In [55]:
station_data = sc.textFile("stations.csv")

stationsHeader = station_data.first()
stations = station_data.filter(lambda row: row != stationsHeader).map(lambda row: row.split(",", -1))

In [63]:
all_distances = stationsInternal = stations.mapPartitions(initStation)

In [64]:
max_dist = all_distances.top(1, key=lambda x: x[2])[0]

In [65]:
print(f'Max distance:\t{max_dist[2]}\nStations:\t{max_dist[0]}-{max_dist[1]}')

Max distance:	37.80477
Stations:	60-Embarcadero at Sansome


3 Найти путь велосипеда с максимальным временем пробега через станции.

In [91]:
id = m[0][0]
print("Велосипед с максимальным временем пробега: ", id)

Велосипед с максимальным временем пробега:  535


In [92]:
tripData.take(5)

['id,duration,start_date,start_station_name,start_station_id,end_date,end_station_name,end_station_id,bike_id,subscription_type,zip_code',
 '4576,63,,South Van Ness at Market,66,8/29/2013 14:14,South Van Ness at Market,66,520,Subscriber,94127',
 '4607,,8/29/2013 14:42,San Jose City Hall,10,8/29/2013 14:43,San Jose City Hall,10,661,Subscriber,95138',
 '4130,71,8/29/2013 10:16,Mountain View City Hall,27,8/29/2013 10:17,Mountain View City Hall,27,48,Subscriber,97214',
 '4251,77,8/29/2013 11:29,San Jose City Hall,10,8/29/2013 11:30,San Jose City Hall,10,26,Subscriber,95060']

In [93]:
# filtering for map
trip_header=tripData.first()

trip=tripData.filter(lambda x: x != trip_header).map(lambda x: x.split(","))

In [95]:
trip_mp = trip.mapPartitions(initTrip)
trip_mp.first()

Trip(trip_id=4130, duration=71, start_date=datetime.datetime(2013, 8, 29, 10, 16), start_station_name='Mountain View City Hall', start_station_id=27, end_date=datetime.datetime(2013, 8, 29, 10, 17), end_station_name='Mountain View City Hall', end_station_id='27', bike_id=48, subscription_type='Subscriber', zip_code='97214')

In [96]:
bike_max_time = trip_mp \
.map(lambda trip: (trip.bike_id, trip.duration)) \
.reduceByKey(lambda a, b: a + b) \
.top(1, key = lambda x: x[1])[0][0]

bike_max_time

535

In [97]:
#trip_header=tripData.first()
#trip=tripData.filter(lambda x: x != trip_header).map(lambda x: x.split(","))


result = trip_mp \
    .filter(lambda x: x.bike_id == bike_max_time) \
    .sortBy(lambda x: x.start_date) \
    .map(lambda x: (x.start_station_name, x.end_station_name)) \
    .first() 

result

('Post at Kearney', 'San Francisco Caltrain (Townsend at 4th)')

4 Найти количество велосипедов в системе.

In [89]:
bikes = tripsInternal.map(lambda row: row.bike_id).distinct()

In [90]:
bikes.count()

700

5 Найти пользователей потративших на поездки более 3 часов.

In [86]:
trips_by_trip_id = tripsInternal.keyBy(lambda trip: trip.trip_id)

In [87]:
trips_by_duration_more_3_hours = trips_by_trip_id.mapValues(lambda trip: trip.duration).filter(lambda trip: trip[1] > 3*60*60)

In [88]:
trips_by_duration_more_3_hours.collect()

[(4639, 11118),
 (4637, 11272),
 (4528, 12280),
 (4363, 15244),
 (4193, 18192),
 (4190, 18240),
 (4225, 21612),
 (4663, 52698),
 (4532, 84990),
 (4521, 85385),
 (5069, 86102),
 (4505, 97713),
 (5539, 10805),
 (6032, 10963),
 (6409, 12408),
 (6408, 12459),
 (5697, 12562),
 (5683, 12875),
 (5218, 13809),
 (5716, 14545),
 (5717, 14568),
 (5241, 15160),
 (5537, 16570),
 (5253, 16713),
 (5531, 16744),
 (6131, 16933),
 (6098, 17593),
 (6091, 17774),
 (5115, 18198),
 (5114, 18206),
 (5250, 18219),
 (5249, 18241),
 (5508, 20920),
 (5506, 20931),
 (5167, 23182),
 (5166, 23212),
 (5640, 23319),
 (5638, 23360),
 (6003, 23694),
 (5380, 25335),
 (5222, 27103),
 (5120, 33088),
 (5113, 34449),
 (5182, 36255),
 (6412, 70731),
 (6178, 70982),
 (6462, 77919),
 (5493, 83224),
 (6282, 241334),
 (7069, 11161),
 (6626, 11632),
 (6963, 11642),
 (7074, 11653),
 (6962, 11675),
 (7067, 11825),
 (6792, 11964),
 (7143, 12303),
 (6613, 12400),
 (7176, 12818),
 (7081, 14180),
 (7079, 14299),
 (7021, 14451),
 (7016,